## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


## Business Understanding

- Predictive sale pricing for relators working with sellers (price the home to sell)

### Problem/Stakeholder
We are a data science consulting company working with a real estate company to help assist with creating a pricing model for incoming listings. 

### Data Understanding

## Data Preparation

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
from random import gauss
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats as stats
from statsmodels.formula.api import ols
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split

%matplotlib inline

Pulling in data and exploring data prior to cleaning.

In [ ]:
data = pd.read_csv('./data/kc_house_data.csv')

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data.info()

We see we have null values in "waterfront", "view", and "yr_renovated" columns. 

In [ ]:
data['price'].describe()

In [ ]:
data.corr()

In [ ]:
sns.heatmap(data.corr());

In [ ]:
#Yr_renovated and price correlation .1296; might flatten past a certain year.  

In [ ]:
data.yr_renovated.value_counts()

We see we have both nulls and "0" values in this column.

In [ ]:
data.yr_renovated.describe()

In [ ]:
17755 - 17011

In [ ]:
data.info()

In [ ]:
data.drop(columns='yr_renovated', inplace=True)

'yr_renovated' only contains 744 values, which is less than 1% of the data. Dropping the column 'yr_renovated' due to the low value counts.

In [ ]:
data.info()

In [ ]:
data.waterfront.value_counts()

In [ ]:
data.view.value_counts()

Only 60 null values, could drop those rows utilizing dropna or replace with our Mode value, "None."

In [ ]:
data['view'].fillna("NONE", inplace=True)

In [ ]:
data['view'].value_counts()

In [ ]:
data.info()

In [ ]:
data['waterfront'].value_counts()

No is overwhelming mode; could replace nulls with mode or create 3rd column "Unknown". 

In [ ]:
data['waterfront'].fillna("UNKNOWN", inplace=True)

In [ ]:
data['waterfront'].value_counts()

In [ ]:
data.info()

In [ ]:
data['sqft_basement'].value_counts()

Here we see we have some values of "?"; we need to decide how to clean this data. We will replace "?" with 0 because a large proportion of our data is at 0.

In [ ]:
data['sqft_basement'].replace(to_replace = '?', value = 0.0, inplace=True)

In [ ]:
data['sqft_basement'].value_counts()

In [ ]:
data['sqft_basement'] = data['sqft_basement'].astype(str).astype(float)

In [ ]:
data['grade'].value_counts()

Since there is only 1 data point with a grade of (3 Poor), we will drop it as in contains <1% of the data.

In [ ]:
data = data[data.grade != '3 Poor']

In [ ]:
data.info()

In [ ]:
data['grade'].value_counts()

In [ ]:
data.corr()

Dropping ID and Date columns as they are not relevant to our analysis.

In [ ]:
data.drop(columns=('id'), inplace=True)

In [ ]:
data.drop(columns=('date'), inplace=True)

Adding price per sqft columnn to help with comparison.

In [ ]:
data['price_per_sqft_living'] = (data['price']/data['sqft_living'])

In [ ]:
data['price_per_sqft_living'].sort_values()

In [ ]:
data['price_per_sqft_living'].describe()

In [ ]:
IQR = 318.33 - 182.29

In [ ]:
IQR * 1.5

In [ ]:
318.33 + 204.06

In [ ]:
(data['price_per_sqft_living'] > 522).value_counts()

In [ ]:
data['price'].describe()

In [ ]:
q3, q1 = np.percentile(data['price'], [75 ,25])
iqr = q3 - q1

In [ ]:
iqr2 = iqr*1.5

In [ ]:
q3 + iqr2

In [ ]:
q1 - iqr2

In [ ]:
(data['price'] > 1129500).value_counts()

In [ ]:
data.head()

We want to further look into how distance from the epicenter of Seattle effects sale price. We will create a column calculating difference between home and Seattle epicenter. We will be utilizing the Seattle Art Museum coordinates for Seattle: (lat = 47.6077, long= -122.337).

In [ ]:
from geopy.distance import geodesic
import geopy

In [ ]:
print(geodesic((47.5112, -122.257), (47.6077, -122.337)).miles)

In [ ]:
data.head()

In [ ]:
coords = (47.6077, -122.337)

In [ ]:
data['distance_from_Seattle'] = data.apply(lambda x: geopy.distance.distance((x.lat, x. long), coords).miles, axis=1)

In [ ]:
data.head()

We have cleaned all of our null values. 

Here, we create visualizations to see if the data is relatively normal and if we want to utilize the data. If we decide to utilize these variables and hot code, we need to drop a column to prevent multicollinearity. Column dropped becomes baseline. We will encode once we train the data. View article link: https://github.com/hoffm386/coefficients-of-dropped-categorical-variables

In [ ]:
data['condition'] = pd.Categorical(data['condition'], ['Poor','Fair','Average','Good', 'Very Good'])

In [ ]:
sns.displot(data=data, x='condition');

In [ ]:
data['grade'] = pd.Categorical(data['grade'], ['3 Poor','4 Low','5 Fair','6 Low Average', '7 Average', '8 Good', '9 Better', '10 Very Good', '11 Excellent', '12 Luxury', '13 Mansion'])

In [ ]:
sns.displot(data=data, x='grade', height=7, aspect=2);

In [ ]:
data.corr()

In [ ]:
cor = data.corr()

plt.figure(figsize = (15, 8))
sns.heatmap(cor, annot=True);

We see that there is a large difference in the correlations between codition and price and grade and price, even though the columns seem to describe similar attributes. 

- 'sqft_living' highly correlated to 'bathrooms', 'grade', 'sqft_above', and 'sqft_living15'
- 'sqft_lot' is highly correlated to 'sqft_lot15'

Train test split. 

In [ ]:
X, y = data.drop(columns='price'), data['price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
training_data, testing_data = train_test_split(data, test_size=0.2, random_state=25)

print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")


In [ ]:
training_data.info()

In [ ]:
testing_data.info()

In [ ]:
X_train.head()

In [ ]:
training_data.head()

In [ ]:
y_train.head()

Above is a sanity check that our training_data lines up with our X_train and our y_train.

We need to deal with our categorical variables, which we will convert utilizing One Hot Encoding. When modeling, we will have to remove one encoder from each category. 

In [ ]:
categoricals = ['view', 'waterfront', 'condition', 'grade']
X_train_dummies = pd.get_dummies(X_train[categoricals], prefix=categoricals)
X_train_dummies

In [ ]:
categoricals = ['view', 'waterfront', 'condition', 'grade']
X_test_dummies = pd.get_dummies(X_test[categoricals], prefix=categoricals)
X_test_dummies

In [ ]:
X_train.info()

Dropping the categorical columns so we can add the dummied columns.

In [ ]:
X_train.drop(['view', 'condition', 'grade', 'waterfront'], axis=1, inplace=True)

In [ ]:
X_train.info()

In [ ]:
X_test.drop(['view', 'condition', 'grade', 'waterfront'], axis=1, inplace=True)

In [ ]:
X_test.info()

In [ ]:
X_test = X_test.join(X_test_dummies)

In [ ]:
X_test.info()

In [ ]:
X_train = X_train.join(X_train_dummies)

In [ ]:
X_train.info()

In [ ]:
training_data = X_train.join(y_train)

In [ ]:
training_data.corr()

### Scaling Data

In [ ]:
ss=StandardScaler()
ss.fit(X_train)
X_train_scaled = ss.transform(X_train)

In [ ]:
X_train_scaled

In [ ]:
lr = LinearRegression()
lr.fit(X_train_scaled,y_train)
lr.score(X_train_scaled, y_train)


In [ ]:
corr = training_data.corr()

In [ ]:
corr

In [ ]:
#remove correlation of features with themselves
corr = corr[corr < 1]

#top positively correlated
corr[corr < 1].unstack().sort_values(ascending=False)[:20]

In [ ]:
corr.unstack().sort_values(ascending=True)[:20]

Shows variables that are highly correlated.

In [ ]:
price_corr = abs(training_data.corr()['price']).sort_values(ascending = False)
price_corr

Arranging correlation of variables to price from highest to lowest based on training data.

# Models

First, we will look at a Dummy Model. 

In [ ]:
dummy_regr = DummyRegressor(strategy="mean")

In [ ]:
dummy_regr.fit(X, y)

In [ ]:
dummy_regr.predict(X)

In [ ]:
dummy_regr.score(X, y)

In [ ]:
plt.hist(data['sqft_living'])

In [ ]:
plt.hist(data['price'])

In [ ]:
plt.hist(data['price_per_sqft_living'])

Starting with our simple model, we will look at the model utilizing sqft_living as our independent variable based on it having the highest correlation with price.

In [ ]:
y = training_data ['price']
X = training_data [['sqft_living']]

lr = sm.OLS(endog=y, exog=sm.add_constant(X))

In [ ]:
simple_model_summary = lr.fit().summary()

In [ ]:
simple_model_summary

For our simple summary, we see an R-squared of .493. Our p-value is <.05, showing that our results are statistically repeatable. 

In [ ]:
#Checking for linear relationship

In [ ]:
plt.scatter(X,y)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
lr = LinearRegression()

lr.fit(X_train, y_train)

train_preds = lr.predict(X_train)
test_preds = lr.predict(X_test)

In [ ]:
train_residuals = y_train - train_preds
test_residuals = y_test - test_preds

In [ ]:
#Checking for normality

In [ ]:
plt.hist(train_residuals, label='Train')
plt.hist(test_residuals, label='Test')

plt.show()

In [ ]:
#Checking for heteroskedasticity

In [ ]:
sns.residplot(x=X_train['sqft_living'], y=y_train)
plt.show()        

In [ ]:
#Our simple model does not show heteroskedasticity and thus is not a good moodel regarding our assumptions.

In [ ]:
#Heteroskadasticity for whole model; not great

In [ ]:
plt.scatter(train_preds, train_residuals, label='Train')

plt.axhline(y=0, color = 'red', label = '0')
plt.xlabel('predictions')
plt.ylabel('residuals')
plt.legend()
plt.show()

In [ ]:
sm.qqplot(train_residuals, line = 'r')

In [ ]:
## Other regressions prior to fitting that checked variables with high correlation to price